# Restaurant Management System

In [1]:
from datetime import date
from datetime import datetime

class Restaurant:
    
    def __init__(self, name, address, tables, id_item_map):
        
        self.name = name.strip()
        self.address = address.strip()
        self.total_tables = tables
        self.items = id_item_map
        self.previous_orders = {}
        self.current_orders = {}
    
    def add_item_menu(self, item):
        self.items[item.id] = item
        
    def get_items(self):
        return [item for item in list(self.items.values()) if item.is_available]
    
    # if it's an prepaid order, add it to previous orders as it's already paid
    # if it's a dine in or carry out order, add it to current orders
    def place_order(self, order):
        if order.prepaid == True:
            self.add_to_previous_order(order)
        else:
            self.current_orders[order.order_id] = order
    
    # if it's an prepaid order, mark the order status as cancelled
    # if it's a dine in or carry out order, remove from the current orders
    def cancel_order(self, order):
        if order.prepaid == True:
            _date = Utils.get_date(order.date)
            
            for prev_order in self.self.previous_orders[_date]:
                if order.order_id == prev_order.order_id:
                    prev_order.status = Status.CANCELLED
                    break
        else:
            order.status = Status.CANCELLED
            self.current_orders.pop(order.order_id)
            self.add_to_previous_order(order)
    
    def get_status(self, order_id):
        if order_id in self.current_orders:
            return self.current_orders[order_id].status
        
        return OrderStatus.CLOSED
        
    def past_orders(self, date=date.today()):
        # key => order_date
        return [order for order in self.previous_orders[date]]
    
    def current_orders(self):
        return list(self.current_orders.values())
    
    def view_menu(self):
        
        print ("{:<3} {:<20} {:<10}".format('SL', 'Name', 'Price'))
        print ("{:<3} {:<20} {:<10}".format('--', '----', '-----'))
        
        for id, item in id_item_map.items():
            print ("{:<3} {:<20} {:<10}".format(id, item.name, item.price))
    
    def search_items(self, name):    
        return [(item.name, item.price) for item in list(self.items.values()) if item.name.lower().__contains__(name.lower())]
    
    
    def add_to_previous_order(self, order):
        
        _date = Utils.get_date(order.date)
        
        if _date not in self.previous_orders:
            self.previous_orders[_date] = []
            
        self.previous_orders[_date].append(order)
        
    # take payment for the current order
    def make_payment(self, order_id, payment):
        
        order = self.current_orders[order_id]
        
        total_amount_before_discount = order.total_amount
        total_amount = order.make_payment(order.payment)
        order.payment = payment
        order.status = Status.CLOSED
        
        # remove the order from current order and add it to previous orders
        if order_id in self.current_orders:
            self.current_orders.pop(order_id)
        
        # add to previous order
        self.add_to_previous_order(order)
        
        # print the bill
        self.print_bill(order)
        
    
    def print_bill(self, order):
        
        print("\nName : {} \t Date : {}".format(order.name, order.date))
        print ("\n{:<3} {:<20} {:<10} {:<5}".format('SL','Item', 'Price', 'Qty'))
        
        i = 0
        for item_id, quant in order.items.items():
            i += 1
            print ("{:<3} {:<20} ${:<10} {:<5}".format(i, id_item_map[item_id].name, id_item_map[item_id].price, quant))
        
        print('\nTotal Bill Amount : ${} | Tax : {} % | Payment Mode: {}'.format(order.total_amount, order.tax_perc, order.payment))
        
        if order.discount_perc != 0:
            print('\nDiscount awarded : {} %'.format(order.discount_perc))
        
        print('******* Have a Great Day !!!*******')
            
        
    def __str__(self):
        return f'Restaurant [name : {self.name}, address : {self.address}, current_orders : {self.current_orders}]'
    
    def __repr__(self):
        return f'Restaurant("name" : {self.name}, "address" : {self.address}, "current_orders" : {self.current_orders})'

In [2]:
class OrderType:
    DINEIN = 'DineIn'
    UBEREATS = 'UberEats'
    GRUBHUB = 'GrubHub'
    DOORDASH = 'DoorDash'
    CARRYOUT = 'CarryOut'

class Status:
    OPEN = 'Open'
    PREPARING = 'Preparing'
    PENDING = 'Pending'
    CLOSED = 'Closed'
    COMPLETED = 'Completed'
    CANCELLED = 'Cancelled'

class Payment:
    CASH = 'Cash'
    ONLINE = 'Online'
    CREDIT = 'Credit'
    DEBIT = 'Debit'
    APPLEPAY = 'ApplePay'
    GOOGLEPAY = 'GooglePay'

class Utils:
    @staticmethod
    def get_date(_datetime):
        
        if isinstance(_datetime, str):
            _datetime = datetime.strptime(_datetime,'%Y-%m-%d %H:%M')
        
        return datetime.date(_datetime)

In [3]:
class Item:
    
    #item_num=0
    
    def __init__(self, id, name, price, is_available=True, quantity=1):
        self.id = id
        self.name = name.strip()
        self.price = float(price)
        #Item.item_num += 1
        #self.id = Item.item_num
        self.quantity = quantity
        self.is_available = is_available
        id_item_map[self.id] = self
    
    def change_item_price(self, item_id, new_price):       
        id_item_map[item_id].price = float(new_price)
        
    def change_item_name(self, id, new_name):
        
        if id in id_item_map:
            old_name = id_item_map[id].name
            id_item_map[id].name = new_name.strip()
            return True
        
        else:
            return False

    def mark_item_unavailable(self, id):
        id_item_map[id].is_available = False
            
    def mark__item_available(self, id):
        id_item_map[id].is_available = True
    
    def __str__(self):
        return f'Item [id : {self.id}, name : {self.name}, price : ${self.price}]'
    
    def __repr__(self):
        return f'Item("id" : {self.id}, "name" : {self.name}, "price" : ${self.price}, "available" : {self.is_available})'

In [4]:
order_map = {}
    
class Order:
    
    order_num = 0
    
    # default date is current date
    def __init__(self, name, order_type, item_quantity_map, order_date):
        
        self.name = name
        self.date = datetime.strptime(order_date, "%Y-%m-%d %H:%M")
        
        # this is to start the order number from 1 on every day, 
        # if (date.today(), 1) not in order_map:
        #Order.order_num  = 1
        #else:
        #  Order.order_num += 1
        
        Order.order_num += 1
        self.order_id = Order.order_num
    
        self.order_type = order_type
        self.items = item_quantity_map
        self.tax_perc = 4
        self.payment = None
        self.prepaid = False
        self.status = Status.OPEN
        self.discount_perc = 0
        self.total_amount = 0
        
        # if the order is ONLINE order, then the payment it's prepaid order
        # status should be closed
        if order_type == OrderType.UBEREATS or order_type == OrderType.GRUBHUB or order_type == OrderType.DOORDASH:
            self.prepaid = True
            self.payment = Payment.ONLINE
            self.status = Status.CLOSED
 
        for item_id, quant in self.items.items():
            item = id_item_map[item_id]
            self.total_amount += quant*item.price
        
        # if it's an prepaid order calculate tax at the time of order
        if self.prepaid:
            self.total_amount = self.make_payment()
        
        self.total_amount = round(self.total_amount,2)
            
    
    # add an item to an existing order and calcuate the charge 
    def add_item(self, item_id, quant=1):
        self.items[item_id] = self.items.get(item_id,0) + quant
        self.total_amount += quant * id_item_map[item_id].price
    
    # remove an item and remove the charge for that
    def remove_item(self, item_id, quant=1):
        if item_id in self.items and self.items[item_id] > 0:
            self.items[item_id] -= 1
            self.total_amount -= quant * id_item_map[item_id].price
            
            # if the number of items ordered is 0, remove from the entry
            if self.items[item_id] == 0:
                self.items.pop(item_id)
    
    # add a discount for an order
    def apply_discount(self, discount_perc):
        self.discount_perc = discount_perc
    
    # check the status of the order
    def get_status(self):
        return self.status
    
    # remove the tax if required
    def remove_tax(self):
        self.tax_perc = 0
    
    # calculate the final bill amount by adding tax and removing discount if any
    # Discount is applied after calculating the tax
    # close the order after payment
    def make_payment(self, payment=Payment.ONLINE):
        
        self.total_amount += (self.total_amount*self.tax_perc)/100
        self.total_amount -= (self.total_amount*self.discount_perc)/100
        self.total_amount = round(self.total_amount, 2)
        
        self.status = Status.CLOSED
        self.payment = payment
        
        return self.total_amount    
    
    def __str__(self):
        return f"""\nName : {self.name} | date : {self.date} | status : {self.status} | prepaid : {self.prepaid} |order_type : {self.order_type} | items_ordered : {self.items} | tax : {self.tax_perc}% | payment : {self.payment} | discount : {self.discount_perc}% | total_amount : ${self.total_amount}"""
    
    def __repr__(self):
        return f"""\nName : {self.name}, date : {self.date}, status : {self.status}, prepaid : {self.prepaid}, order_type : {self.order_type},items_ordered : {self.items}, tax : {self.tax_perc}%, payment : {self.payment}, discount : {self.discount_perc}%, total_amount : ${self.total_amount}"""
        

In [6]:
id_item_map = {}

item1 = Item(1,'Chicken Biriyani', 18)
item2 = Item(2,'Lamb Biriyani', 22)
item3 = Item(3,'Goat Biriyani', 21)
item4 = Item(4,'Fish Biriyani', 22)
item5 = Item(5,'Shrimp Biriyani', 23)
item6 = Item(6,'Vegetable Biriyani', 16)
item7 = Item(7,'Lamb Karahi', 17.99)
item8 = Item(8,'Goat Curry', 17.99)
item9 = Item(9,'Fish Curry', 20, False)
item10 = Item(10,'Shrimp Karahi', 20.99)
item11 = Item(11,'Chicken Tikka Masala', 17)
item12 = Item(12,'Water Bottle', 1)
item13 = Item(13,'Coke', 2)
item14 = Item(14,'Sprite', 2)
item15 = Item(15,'ThumsUp', 2)
item16 = Item(16,'Mango Lassi', 4)
item17 = Item(17,'Chicken Mango Curry', 17.99)
item18 = Item(18,'Goat Karahi', 17.99)
item19 = Item(19,'Tandoori Salmon', 22.99, False)
item20 = Item(20,'Tandoori Shrimp', 22.99, False)
item21 = Item(21,'Naan', 1.99)
item22 = Item(22,'Garlic Naan', 2.99)
item23 = Item(23,'BUtter Naan', 2.99)
item24 = Item(24,'Roti', 2.99)
item25 = Item(25,'Samosa', 2.99, True, 2)
item26 = Item(26,'Appetizers', 4.99)
item27 = Item(27,'Dal Makhani', 16.99)
item28 = Item(28,'Chana Masala', 14.99)
item29 = Item(29,'Saag Paneer', 16.99)
item30 = Item(30,'Ice Cream', 3.99)

In [7]:
import unittest

testRestaurant = Restaurant('Test Restaurant', 'xyz Street', 4, id_item_map)

class OrderTests(unittest.TestCase):
    
    def test_order_id(self):
        test_date = '2022-11-06 11:40'
        order1 = Order('Mokshit', OrderType.UBEREATS, { 1 : 1, 12 : 1}, test_date)
        testRestaurant.place_order(order1)
        self.assertEqual(order1.order_id, testRestaurant.previous_orders[Utils.get_date(test_date)][0].order_id,'Id should Match')
        
    def test_tax_perc(self):
        test_date = '2022-11-06 11:40'
        
        order1 = Order('Mokshit', OrderType.UBEREATS, { 1 : 1, 12 : 1}, test_date)
        # order placed above
        #testRestaurant.place_order(order1)
        self.assertEqual(order1.tax_perc, testRestaurant.previous_orders[Utils.get_date(test_date)][0].tax_perc,'Tax perc should match')

if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.005s

OK


In [9]:
restaurant = Restaurant('ABC Cuisine', '555 XYZ Road, MD', 10, id_item_map)

In [10]:
restaurant.view_menu()

SL  Name                 Price     
--  ----                 -----     
1   Chicken Biriyani     18.0      
2   Lamb Biriyani        22.0      
3   Goat Biriyani        21.0      
4   Fish Biriyani        22.0      
5   Shrimp Biriyani      23.0      
6   Vegetable Biriyani   16.0      
7   Lamb Karahi          17.99     
8   Goat Curry           17.99     
9   Fish Curry           20.0      
10  Shrimp Karahi        20.99     
11  Chicken Tikka Masala 17.0      
12  Water Bottle         1.0       
13  Coke                 2.0       
14  Sprite               2.0       
15  ThumsUp              2.0       
16  Mango Lassi          4.0       
17  Chicken Mango Curry  17.99     
18  Goat Karahi          17.99     
19  Tandoori Salmon      22.99     
20  Tandoori Shrimp      22.99     
21  Naan                 1.99      
22  Garlic Naan          2.99      
23  BUtter Naan          2.99      
24  Roti                 2.99      
25  Samosa               2.99      
26  Appetizers           4.9

In [11]:
# view the item specials
print(restaurant.search_items('chicken'))

[('Chicken Biriyani', 18.0), ('Chicken Tikka Masala', 17.0), ('Chicken Mango Curry', 17.99)]


In [12]:
# item_quantity map => { item_id : quantity}

order1 = Order('Mokshit', OrderType.UBEREATS, { 1 : 1, 12 : 1}, '2022-11-06 14:30')
order2 = Order('Ram', OrderType.DINEIN, { 21 : 3, 3 : 2, 12 : 1, 14: 1}, '2022-11-11 20:10')
order3 = Order('Madhav', OrderType.DOORDASH, { 4 : 1, 15: 1}, '2022-11-08 10:30')
order4 = Order('Sree', OrderType.GRUBHUB, { 6 : 1, 29 : 1, 23: 1}, '2022-11-10 11:20')
order5 = Order('Hari', OrderType.CARRYOUT, { 9 : 1, 17 : 1}, '2022-11-17 21:20')

In [13]:
# place the orders
restaurant.place_order(order1)
restaurant.place_order(order2)
restaurant.place_order(order3)
restaurant.place_order(order4)
restaurant.place_order(order5)

In [14]:
print('Current Orders ',restaurant.current_orders)
print('\nPrevious Orders ',restaurant.previous_orders)

Current Orders  {4: 
Name : Ram, date : 2022-11-11 20:10:00, status : Open, prepaid : False, order_type : DineIn,items_ordered : {21: 3, 3: 2, 12: 1, 14: 1}, tax : 4%, payment : None, discount : 0%, total_amount : $50.97, 7: 
Name : Hari, date : 2022-11-17 21:20:00, status : Open, prepaid : False, order_type : CarryOut,items_ordered : {9: 1, 17: 1}, tax : 4%, payment : None, discount : 0%, total_amount : $37.99}

Previous Orders  {datetime.date(2022, 11, 6): [
Name : Mokshit, date : 2022-11-06 14:30:00, status : Closed, prepaid : True, order_type : UberEats,items_ordered : {1: 1, 12: 1}, tax : 4%, payment : Online, discount : 0%, total_amount : $19.76], datetime.date(2022, 11, 8): [
Name : Madhav, date : 2022-11-08 10:30:00, status : Closed, prepaid : True, order_type : DoorDash,items_ordered : {4: 1, 15: 1}, tax : 4%, payment : Online, discount : 0%, total_amount : $24.96], datetime.date(2022, 11, 10): [
Name : Sree, date : 2022-11-10 11:20:00, status : Closed, prepaid : True, order_t

In [15]:
# remove an item
order2.remove_item(14)

order5.make_payment(Payment.DEBIT)

print('Current Orders ',restaurant.current_orders)

Current Orders  {4: 
Name : Ram, date : 2022-11-11 20:10:00, status : Open, prepaid : False, order_type : DineIn,items_ordered : {21: 3, 3: 2, 12: 1}, tax : 4%, payment : None, discount : 0%, total_amount : $48.97, 7: 
Name : Hari, date : 2022-11-17 21:20:00, status : Closed, prepaid : False, order_type : CarryOut,items_ordered : {9: 1, 17: 1}, tax : 4%, payment : Debit, discount : 0%, total_amount : $39.51}


In [16]:
import unittest

class ItemTests(unittest.TestCase):
    
    def test_item_id(self):
        item1 = Item(101,'Test item 1',10)
        self.assertEqual(item1.id, id_item_map[item1.id].id,'Id should Match')
        
    def test_item_price(self):
        item2 = Item(102,'Test item 2',10)
        item2.change_item_price(item2.id, 12)
        self.assertEqual(item2.price, id_item_map[item2.id].price,'Change Price should match')
    
    def test_name_change(self):
        new_name = 'Test Item change 2'
        item2.change_item_name(2,new_name)
        self.assertEqual(new_name, id_item_map[item2.id].name, 'Changed item name should be available')
    
    def test_item_availability(self):
        self.assertTrue(id_item_map[item1.id].is_available, 'Item should be available')
    
    def test_item_availability_False(self):
        item3 = Item(103,'Test item 3',10)
        item3.mark_item_unavailable(item3.id)
        self.assertFalse(id_item_map[item3.id].is_available, 'Item should not be available')
    

if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....F.
FAIL: test_order_id (__main__.OrderTests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_6016\4078217301.py", line 11, in test_order_id
    self.assertEqual(order1.order_id, testRestaurant.previous_orders[Utils.get_date(test_date)][0].order_id,'Id should Match')
AssertionError: 8 != 1 : Id should Match

----------------------------------------------------------------------
Ran 7 tests in 0.009s

FAILED (failures=1)
